In [1]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [2]:
def bfi_calc(df,filename):
    # Define the scoring keys
    scoring_key = {
        'Extraversion': [1, 6, 11, 16, 21, 26, 31, 36],
        'Agreeableness': [2, 7, 12, 17, 22, 27, 32, 37, 42],
        'Conscientiousness': [3, 8, 13, 18, 23, 28, 33, 38, 43],
        'Neuroticism': [4, 9, 14, 19, 24, 29, 34, 39],
        'Openness': [5, 10, 15, 20, 25, 30, 35, 40, 41, 44]
    }
    
    # Define reverse-scored items
    reverse_scored = [6, 21, 31, 2, 12, 27, 37, 8, 18, 23, 43, 9, 24, 34, 35, 41]
    reverse_scored = [6, 21, 31, 2, 12, 27, 37, 8, 18, 23, 43, 9, 24, 34, 35, 41]
    
    # Function to adjust scores
    def adjust_score(row):
        #print("for this row", row['persona'],"$$$$$", row['Itemnum'])
        if row['Itemnum'] in reverse_scored:
            #print("hit")
            return 6 - row['answer']
        return row['answer']
    
    # Add adjusted score column
    df['adjusted_score'] = df.apply(adjust_score,axis=1)
    #print(df[:44])
    # Function to calculate trait score
    def calculate_trait_score(persona_data, trait_items):
        scores = []
        for item in trait_items:
            #print("§§§§§§§§§§§§§§§§§§§§")
            score = persona_data.loc[persona_data['Itemnum'] == item, 'answer'].iloc[0]
            if item in reverse_scored:
                score = 6 - score  # Reverse the score
            scores.append(score)
        return np.mean(scores)
    
    # Group by persona and calculate scores
    results = []
    for persona, group in df.groupby('persona'):
        scores = {}
        for trait, items in scoring_key.items():
            scores[trait] = calculate_trait_score(group, items)
        results.append({'Persona': persona, **scores})
    
    # Create a dataframe with the results
    results_df = pd.DataFrame(results)
    results_df=results_df.round(3)
    # Display the results
    print(results_df)
    
    results_df.to_csv(filename, index=False)

#### Calculating results for Initial Criminal list (22)

In [4]:
df = pd.read_csv('results2.csv')

In [5]:
df2 = df.copy()

In [6]:
df2

,persona,text,likert_scale,Itemnum,answer
0,Saddam Hussein,Is talkative,1 to 5,itemcount,5
1,Saddam Hussein,Tends to find fault with others,1 to 5,itemcount,4
2,Saddam Hussein,"Is depressed, blue",1 to 5,itemcount,5
3,Saddam Hussein,"Is original, comes up with new ideas",1 to 5,itemcount,4
4,Saddam Hussein,"Is relaxed, handles stress well",1 to 5,itemcount,5
...,...,...,...,...,...
2592,Ted Kaczynski,"Likes to reflect, play with ideas",1 to 5,40,4
2593,Ted Kaczynski,Has few artistic interests,1 to 5,41,3
2594,Ted Kaczynski,Likes to cooperate with others,1 to 5,42,2
2595,Ted Kaczynski,Is easily distracted,1 to 5,43,5


In [7]:
df3=df2[1629:2598]

In [8]:
df3

,persona,text,likert_scale,Itemnum,answer
1629,Saddam Hussein,Is talkative,1 to 5,1,5
1630,Saddam Hussein,Tends to find fault with others,1 to 5,2,5
1631,Saddam Hussein,Does a thorough job,1 to 5,3,3
1632,Saddam Hussein,"Is depressed, blue",1 to 5,4,1
1633,Saddam Hussein,"Is original, comes up with new ideas",1 to 5,5,4
...,...,...,...,...,...
2592,Ted Kaczynski,"Likes to reflect, play with ideas",1 to 5,40,4
2593,Ted Kaczynski,Has few artistic interests,1 to 5,41,3
2594,Ted Kaczynski,Likes to cooperate with others,1 to 5,42,2
2595,Ted Kaczynski,Is easily distracted,1 to 5,43,5


In [9]:
print(df3['Itemnum'].dtype)  # Should show 'object'

# Attempt to convert 'itemnum' to integer
try:
    df3['Itemnum'] = df3['Itemnum'].astype(int)
except ValueError:
    # If there's an error, it might be due to non-numeric values
    print("Conversion failed due to non-numeric values.")
    
    # Convert 'itemnum' to numeric, setting errors='coerce' will replace non-numeric values with NaN
    df3['Itemnum'] = pd.to_numeric(df3['Itemnum'], errors='coerce')

    # Check for rows where conversion to int failed
    print(df3[df3['Itemnum'].isna()])

    # Optionally drop or handle NaN values
    # data.dropna(subset=['itemnum'], inplace=True)

# Check the data type after conversion
print(df3['Itemnum'].dtype)

object
int64


C:\Users\PC\AppData\Local\Temp\ipykernel_13948\4226010020.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Itemnum'] = df3['Itemnum'].astype(int)


In [10]:
df3.dtypes

persona         object
text            object
likert_scale    object
Itemnum          int64
answer           int64
dtype: object

In [11]:
import pandas as pd
import numpy as np

# Read the CSV file
df=df3.copy()
# Define the scoring keys
scoring_key = {
    'Extraversion': [1, 6, 11, 16, 21, 26, 31, 36],
    'Agreeableness': [2, 7, 12, 17, 22, 27, 32, 37, 42],
    'Conscientiousness': [3, 8, 13, 18, 23, 28, 33, 38, 43],
    'Neuroticism': [4, 9, 14, 19, 24, 29, 34, 39],
    'Openness': [5, 10, 15, 20, 25, 30, 35, 40, 41, 44]
}

# Define reverse-scored items
reverse_scored = [6, 21, 31, 2, 12, 27, 37, 8, 18, 23, 43, 9, 24, 34, 35, 41]
reverse_scored = [6, 21, 31, 2, 12, 27, 37, 8, 18, 23, 43, 9, 24, 34, 35, 41]

# Function to adjust scores
def adjust_score(row):
    #print("for this row", row['persona'],"$$$$$", row['Itemnum'])
    if row['Itemnum'] in reverse_scored:
        #print("hit")
        return 6 - row['answer']
    return row['answer']

# Add adjusted score column
df['adjusted_score'] = df.apply(adjust_score,axis=1)
#print(df[:44])
# Function to calculate trait score
def calculate_trait_score(persona_data, trait_items):
    scores = []
    for item in trait_items:
        #print("§§§§§§§§§§§§§§§§§§§§")
        score = persona_data.loc[persona_data['Itemnum'] == item, 'answer'].iloc[0]
        if item in reverse_scored:
            score = 6 - score  # Reverse the score
        scores.append(score)
    return np.mean(scores)

# Group by persona and calculate scores
results = []
for persona, group in df.groupby('persona'):
    scores = {}
    for trait, items in scoring_key.items():
        scores[trait] = calculate_trait_score(group, items)
    results.append({'Persona': persona, **scores})

# Create a dataframe with the results
results_df = pd.DataFrame(results)

# Display the results
print(results_df)
#results_df.to_csv('bfi_scores.csv', index=False)

                 Persona  Extraversion  Agreeableness  Conscientiousness  \
0           Adolf Hitler         3.000       2.777778           2.555556   
1              Al Capone         2.875       2.666667           3.777778   
2       Benito Mussolini         3.750       2.888889           3.000000   
3          Bonnie Parker         2.750       2.777778           2.666667   
4         Charles Manson         2.375       2.666667           3.777778   
5           Fidel Castro         2.500       3.000000           3.222222   
6         Harold Shipman         2.875       3.333333           3.000000   
7               Idi Amin         3.375       2.555556           3.111111   
8         Jeffrey Dahmer         3.000       3.444444           3.333333   
9        John Wayne Gacy         3.625       3.111111           2.444444   
10          Josef Stalin         3.500       2.888889           3.111111   
11           Kim Il Sung         3.250       2.333333           4.555556   
12          

In [13]:
results_df.round(3)

,Persona,Extraversion,Agreeableness,Conscientiousness,Neuroticism,Openness
0,Adolf Hitler,3.000,2.778,2.556,2.875,2.6
1,Al Capone,2.875,2.667,3.778,2.875,3.3
2,Benito Mussolini,3.750,2.889,3.000,3.000,3.3
3,Bonnie Parker,2.750,2.778,2.667,2.625,3.5
4,Charles Manson,2.375,2.667,3.778,3.625,3.2
5,Fidel Castro,2.500,3.000,3.222,2.875,3.3
6,Harold Shipman,2.875,3.333,3.000,2.375,2.9
7,Idi Amin,3.375,2.556,3.111,3.375,3.5
8,Jeffrey Dahmer,3.000,3.444,3.333,3.000,3.5
9,John Wayne Gacy,3.625,3.111,2.444,3.125,3.5


#### Results for Peace prize Winners

In [34]:
df_good=pd.read_csv('results3.csv')

print(df_good.head())

df_good.groupby(['persona']).count()

          persona                                  text likert_scale  Itemnum  \
0  Nelson Mandela                          Is talkative       1 to 5        1   
1  Nelson Mandela       Tends to find fault with others       1 to 5        2   
2  Nelson Mandela                   Does a thorough job       1 to 5        3   
3  Nelson Mandela                    Is depressed, blue       1 to 5        4   
4  Nelson Mandela  Is original, comes up with new ideas       1 to 5        5   

   answer  
0       3  
1       3  
2       4  
3       3  
4       4  


,text,likert_scale,Itemnum,answer
persona,,,,
Ai Weiwei,44,44,44,44
Andrei Sakharov,44,44,44,44
Aung San Suu Kyi,44,44,44,44
Chen Guangcheng,44,44,44,44
Denis Mukwege,44,44,44,44
Desmond Tutu,44,44,44,44
Dolores Huerta,44,44,44,44
Eleanor Roosevelt,44,44,44,44
Elie Wiesel,44,44,44,44


In [27]:
df_good.dtypes

persona         object
text            object
likert_scale    object
Itemnum          int64
answer           int64
dtype: object

In [32]:
bfi_calc(df_good,"bfi_good.csv")

                   Persona  Extraversion  Agreeableness  Conscientiousness  \
0                Ai Weiwei         3.000          3.000              2.667   
1          Andrei Sakharov         2.500          4.000              2.556   
2         Aung San Suu Kyi         3.500          2.556              2.889   
3          Chen Guangcheng         2.625          3.111              2.556   
4            Denis Mukwege         2.000          3.222              2.778   
5             Desmond Tutu         3.375          3.444              3.000   
6           Dolores Huerta         2.375          2.889              2.889   
7        Eleanor Roosevelt         3.750          3.889              3.778   
8              Elie Wiesel         3.250          3.000              2.778   
9              Harvey Milk         2.875          2.889              3.222   
10            Ida B. Wells         3.625          3.000              3.556   
11              Liu Xiaobo         2.875          2.444         

#### Results for Authors

In [3]:
df_auth=pd.read_csv('results4.csv')
df_auth.head()

,persona,text,likert_scale,Itemnum,answer
0,Hans Christian Andersen,Is talkative,1 to 5,1,3
1,Hans Christian Andersen,Tends to find fault with others,1 to 5,2,3
2,Hans Christian Andersen,Does a thorough job,1 to 5,3,3
3,Hans Christian Andersen,"Is depressed, blue",1 to 5,4,2
4,Hans Christian Andersen,"Is original, comes up with new ideas",1 to 5,5,5


In [4]:
df_auth.dtypes

persona         object
text            object
likert_scale    object
Itemnum          int64
answer           int64
dtype: object

In [5]:
bfi_calc(df_auth,"bfi_authors.csv")

                    Persona  Extraversion  Agreeableness  Conscientiousness  \
0         Alexander Pushkin         2.500          2.889              3.556   
1               Andy Warhol         2.750          3.000              3.000   
2           Charles Dickens         3.625          3.000              2.778   
3          Federico Fellini         3.000          2.667              3.222   
4               George Sand         3.125          3.222              3.444   
5   Hans Christian Andersen         3.375          3.222              3.333   
6             J. K. Rowling         3.500          3.333              2.556   
7          J. R. R. Tolkien         3.375          3.000              3.444   
8               James Joyce         3.375          3.111              3.333   
9       John Maynard Keynes         2.000          2.667              2.667   
10              John McCain         3.250          3.333              2.556   
11            Leonard Cohen         3.375          3

#### Results for TV actors

In [6]:
df_tva=pd.read_csv('results5.csv')
df_tva.head()

,persona,text,likert_scale,Itemnum,answer
0,Ronald Reagan,Is talkative,1 to 5,1,1
1,Ronald Reagan,Tends to find fault with others,1 to 5,2,3
2,Ronald Reagan,Does a thorough job,1 to 5,3,3
3,Ronald Reagan,"Is depressed, blue",1 to 5,4,2
4,Ronald Reagan,"Is original, comes up with new ideas",1 to 5,5,4


In [7]:
df_tva.dtypes

persona         object
text            object
likert_scale    object
Itemnum          int64
answer           int64
dtype: object

In [8]:
bfi_calc(df_tva,"bfi_actors.csv")

               Persona  Extraversion  Agreeableness  Conscientiousness  \
0            Al Pacino         3.375          3.333              3.556   
1          Alain Delon         3.125          3.000              2.889   
2            Brad Pitt         3.250          3.111              2.889   
3       Bradley Cooper         2.000          2.667              3.222   
4            Bruce Lee         3.000          2.667              3.000   
5   Charlotte Rampling         3.250          3.000              3.444   
6          Corbin Bleu         3.125          3.333              3.222   
7     Elizabeth Taylor         3.375          3.444              3.000   
8         Hebe Camargo         3.375          3.222              2.667   
9          Johnny Depp         3.000          2.444              3.333   
10        Judy Garland         3.250          3.667              3.333   
11   Leonardo DiCaprio         3.250          3.333              3.111   
12    Marlene Dietrich         3.000  

#### Results for Athletes

In [9]:
df_ath=pd.read_csv('results6.csv')
df_ath.head()

,persona,text,likert_scale,Itemnum,answer
0,Jet Li,Is talkative,1 to 5,1,1
1,Jet Li,Tends to find fault with others,1 to 5,2,4
2,Jet Li,Does a thorough job,1 to 5,3,5
3,Jet Li,"Is depressed, blue",1 to 5,4,4
4,Jet Li,"Is original, comes up with new ideas",1 to 5,5,3


In [10]:
df_ath.dtypes

persona         object
text            object
likert_scale    object
Itemnum          int64
answer           int64
dtype: object

In [11]:
bfi_calc(df_ath,"bfi_athletes.csv")

                     Persona  Extraversion  Agreeableness  Conscientiousness  \
0                  Attalus I         3.250          3.111              3.000   
1            Bernard Hinault         2.500          2.889              3.556   
2              Dave Bautista         2.875          3.111              3.556   
3               Dick Fosbury         2.875          3.889              3.222   
4                    Eusébio         3.375          3.667              2.889   
5   Florence Griffith Joyner         2.625          2.556              2.444   
6            Hana Mandlíková         2.500          3.444              3.000   
7       Jackie Joyner-Kersee         3.750          3.556              2.778   
8                     Jet Li         2.375          2.778              2.889   
9           Magdalena Neuner         3.125          2.333              2.556   
10           Magnús Scheving         3.875          4.222              3.111   
11             Matti Nykänen         3.0

#### Results for Scientists

In [20]:
df_sci=pd.read_csv('results7.csv')
df_sci.head()

,persona,text,likert_scale,Itemnum,answer
0,Leonardo da Vinci,Is talkative,1 to 5,1,5
1,Leonardo da Vinci,Tends to find fault with others,1 to 5,2,3
2,Leonardo da Vinci,Does a thorough job,1 to 5,3,2
3,Leonardo da Vinci,"Is depressed, blue",1 to 5,4,4
4,Leonardo da Vinci,"Is original, comes up with new ideas",1 to 5,5,5


In [21]:
df_sci.dtypes

persona         object
text            object
likert_scale    object
Itemnum          int64
answer           int64
dtype: object

In [22]:
bfi_calc(df_sci,"bfi_scientists.csv")

                    Persona  Extraversion  Agreeableness  Conscientiousness  \
0      Carl Friedrich Gauss         2.875          3.222              3.778   
1        Christiaan Huygens         2.875          4.000              3.444   
2             Francis Bacon         2.250          3.000              3.444   
3           Galileo Galilei         3.375          3.111              3.556   
4         Guglielmo Marconi         3.500          2.111              3.333   
5    Heike Kamerlingh Onnes         2.750          3.444              3.444   
6       Hildegard of Bingen         2.500          2.778              3.000   
7        John Amos Comenius         3.875          3.222              3.000   
8                John Locke         2.875          2.889              3.222   
9          John von Neumann         3.000          2.556              3.778   
10  Konstantin Tsiolkovskii         3.750          2.889              3.778   
11               Kurt Gödel         3.375          2

#### Miscellaneous cells

In [2]:
df = pd.read_csv('results2.csv')

In [3]:
df_comb=df[667:]

In [4]:
df_comb.head(10)

,persona,text,likert_scale,Itemnum,answer
667,Saddam Hussein,Is talkative,1 to 5,1,1
668,Saddam Hussein,Tends to find fault with others,1 to 5,2,4
669,Saddam Hussein,Does a thorough job,1 to 5,3,1
670,Saddam Hussein,"Is depressed, blue",1 to 5,4,1
671,Saddam Hussein,"Is original, comes up with new ideas",1 to 5,5,5
672,Saddam Hussein,Is reserved,1 to 5,6,5
673,Saddam Hussein,Is helpful and unselfish with others,1 to 5,7,3
674,Saddam Hussein,Can be somewhat careless,1 to 5,8,3
675,Saddam Hussein,"Is relaxed, handles stress well",1 to 5,9,5
676,Saddam Hussein,Is curious about many different things,1 to 5,10,3


In [5]:
df_combined = df_comb.groupby(['persona', 'Itemnum'])['answer'].agg(['mean'])

In [6]:
df_combined

mean
persona       Itemnum      
Adolf Hitler  1         2.5
              10        3.5
              11        3.5
              12        4.0
              13        4.0
...                     ...
Ted Kaczynski 5         3.5
              6         3.5
              7         4.5
              8         4.0
              9         5.0

[968 rows x 1 columns]

In [18]:
print(df_comb['persona'].nunique() * df_comb['Itemnum'].nunique())

968


In [19]:
df_combined.index

MultiIndex([( 'Adolf Hitler',  '1'),
            ( 'Adolf Hitler', '10'),
            ( 'Adolf Hitler', '11'),
            ( 'Adolf Hitler', '12'),
            ( 'Adolf Hitler', '13'),
            ( 'Adolf Hitler', '14'),
            ( 'Adolf Hitler', '15'),
            ( 'Adolf Hitler', '16'),
            ( 'Adolf Hitler', '17'),
            ( 'Adolf Hitler', '18'),
            ...
            ('Ted Kaczynski', '40'),
            ('Ted Kaczynski', '41'),
            ('Ted Kaczynski', '42'),
            ('Ted Kaczynski', '43'),
            ('Ted Kaczynski', '44'),
            ('Ted Kaczynski',  '5'),
            ('Ted Kaczynski',  '6'),
            ('Ted Kaczynski',  '7'),
            ('Ted Kaczynski',  '8'),
            ('Ted Kaczynski',  '9')],
           names=['persona', 'Itemnum'], length=968)

In [15]:
df_combined.to_clipboard()

In [51]:


data = df3
# List of item numbers for each BFI trait and reverse scored items
extraversion_items = [1, 6, 11, 16, 21, 26, 31, 36]
agreeableness_items = [2, 7, 12, 17, 22, 27, 32, 37, 42]
conscientiousness_items = [3, 8, 13, 18, 23, 28, 33, 38, 43]
neuroticism_items = [4, 9, 14, 19, 24, 29, 34, 39]
openness_items = [5, 10, 15, 20, 25, 30, 35, 40, 41, 44]

reverse_scored_items = [6, 21, 31, 2, 12, 27, 37, 42, 8, 18, 23, 43, 9, 24, 34, 35, 41]

# Function to adjust for reverse scored items
def adjust_reverse_score(row):
    if row['Itemnum'] in reverse_scored_items:
        return 6 - row['answer']
        print("hit")
    else:
        return row['answer']


# Apply the reverse scoring adjustment
data['adjusted_score'] = data.apply(adjust_reverse_score, axis=1)
print(data[0:44])


             persona                                           text  \
1629  Saddam Hussein                                   Is talkative   
1630  Saddam Hussein                Tends to find fault with others   
1631  Saddam Hussein                            Does a thorough job   
1632  Saddam Hussein                             Is depressed, blue   
1633  Saddam Hussein           Is original, comes up with new ideas   
1634  Saddam Hussein                                    Is reserved   
1635  Saddam Hussein           Is helpful and unselfish with others   
1636  Saddam Hussein                       Can be somewhat careless   
1637  Saddam Hussein                Is relaxed, handles stress well   
1638  Saddam Hussein         Is curious about many different things   
1639  Saddam Hussein                              Is full of energy   
1640  Saddam Hussein                    Starts quarrels with others   
1641  Saddam Hussein                           Is a reliable worker   
1642  

C:\Users\PC\AppData\Local\Temp\ipykernel_10320\3711336549.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['adjusted_score'] = data.apply(adjust_reverse_score, axis=1)


In [78]:
# Check the shape of the dataframe
print(df.shape)
print("§§§§§§§§§§§§§§§§§§§§§§§§§§§")
# Check if we have 22 unique personas
print(df['persona'].nunique())
print("§§§§§§§§§§§§§§§§§§§§§§§§§§§")
# Check if each persona has 44 items
print(df.groupby('persona')['Itemnum'].nunique().value_counts())
print("§§§§§§§§§§§§§§§§§§§§§§§§§§§")
# Check the range of itemnum
print(df['Itemnum'].min(), df['Itemnum'].max())
print("§§§§§§§§§§§§§§§§§§§§§§§§§§§")
# Check the range of answer values
print(df['answer'].min(), df['answer'].max())
print("§§§§§§§§§§§§§§§§§§§§§§§§§§§")
# Display a sample of the data
print(df.head(10))

(968, 5)
§§§§§§§§§§§§§§§§§§§§§§§§§§§
22
§§§§§§§§§§§§§§§§§§§§§§§§§§§
Itemnum
44    22
Name: count, dtype: int64
§§§§§§§§§§§§§§§§§§§§§§§§§§§
1 9
§§§§§§§§§§§§§§§§§§§§§§§§§§§
1 6
§§§§§§§§§§§§§§§§§§§§§§§§§§§
             persona                                    text likert_scale  \
1629  Saddam Hussein                            Is talkative       1 to 5   
1630  Saddam Hussein         Tends to find fault with others       1 to 5   
1631  Saddam Hussein                     Does a thorough job       1 to 5   
1632  Saddam Hussein                      Is depressed, blue       1 to 5   
1633  Saddam Hussein    Is original, comes up with new ideas       1 to 5   
1634  Saddam Hussein                             Is reserved       1 to 5   
1635  Saddam Hussein    Is helpful and unselfish with others       1 to 5   
1636  Saddam Hussein                Can be somewhat careless       1 to 5   
1637  Saddam Hussein         Is relaxed, handles stress well       1 to 5   
1638  Saddam Hussein  Is cu

In [62]:
(data['persona'][1])

KeyError: 1

In [63]:
type(data)

pandas.core.frame.DataFrame

In [36]:
# Creating a pivot table
pivot_table = data.pivot(index='persona', columns='Itemnum', values='adjusted_score')
print(pivot_table)

Itemnum               1  10  11  12  13  14  15  16  17  18  ...  40  41  42  \
persona                                                      ...               
Adolf Hitler          4   4   5   5   3   3   3   1   3   2  ...   3   4   5   
Al Capone             3   3   5   5   3   5   3   4   3   1  ...   5   3   2   
Benito Mussolini      3   4   3   3   4   3   3   5   3   3  ...   4   1   3   
Bonnie Parker         2   2   4   4   5   4   4   2   3   3  ...   3   5   3   
Charles Manson        2   4   2   5   4   1   2   5   4   5  ...   5   4   5   
Fidel Castro          4   1   1   3   5   3   3   3   3   4  ...   3   3   2   
Harold Shipman        3   1   1   1   5   4   3   2   5   4  ...   3   3   1   
Idi Amin              3   1   4   5   5   3   4   4   3   3  ...   3   3   1   
Jeffrey Dahmer        1   5   4   4   5   4   5   2   3   5  ...   2   5   5   
John Wayne Gacy       3   5   3   2   5   5   1   5   1   5  ...   5   5   4   
Josef Stalin          5   3   1   3   4 

In [37]:

# Summing scores for each BFI trait
pivot_table['Extraversion'] = pivot_table[extraversion_items].sum(axis=1)
pivot_table['Agreeableness'] = pivot_table[agreeableness_items].sum(axis=1)
pivot_table['Conscientiousness'] = pivot_table[conscientiousness_items].sum(axis=1)
pivot_table['Neuroticism'] = pivot_table[neuroticism_items].sum(axis=1)
pivot_table['Openness'] = pivot_table[openness_items].sum(axis=1)

# Selecting only the BFI scores columns
bfi_scores = pivot_table[['Extraversion', 'Agreeableness', 'Conscientiousness', 'Neuroticism', 'Openness']].reset_index()

# Display the BFI scores
print(bfi_scores)

KeyError: "None of [Index([1, 6, 11, 16, 21, 26, 31, 36], dtype='int64', name='Itemnum')] are in the [columns]"